In [1]:
import numpy as np
from scipy import sparse as sp

from src.hyperparameters_optimizer import HyperparametersOptimizer
from src.recommender_model import RecommenderModel
from src.utils import train_model, write_submission

# EASE$^{\text{R}}$
This notebook provides an implementation of the EASE$^{\text{R}}$ model. See [Embarrassingly Shallow Autoencoders for Sparse Data, Harald Steck](https://arxiv.org/pdf/1905.03375).

In [3]:
class EASER(RecommenderModel):
	"""
	Important note:
		On this dataset with a URM about the size of (35000,40000), this needs at least 60GB of RAM.
		Run on g-colab TPUs to get 300+GB of RAM.
	"""
	def __init__(self):
		super(EASER, self).__init__()
		self.lambda_reg: float = 0

	def fit(self, urm: sp.csr_matrix, lambda_reg: float = 200, **kwargs) -> None:
		self.urm = urm.astype(np.float32)
		self.lambda_reg = lambda_reg

		g = (self.urm.T @ self.urm)
		g += sp.identity(g.shape[0], dtype=np.float32) * self.lambda_reg
		g = g.toarray().astype(np.float32)
		p = np.linalg.inv(g)
		b = p / (-np.diag(p))
		np.fill_diagonal(b, 0.)

		self.urm_pred = self.urm @ b

In [4]:
optimizer = HyperparametersOptimizer({
	'lambda_reg': [40, 45, 50, 55, 60],
}, EASER)
_, best_parameters = optimizer.optimize()

100%|██████████| 5/5 [18:46<00:00, 225.26s/it, Best MAP@10: 0.0776 with ['lambda_reg: 4.50e+01']]


In [5]:
ease_r_submission, _ = train_model(EASER(), test_size=0, **best_parameters)
write_submission(ease_r_submission, "ease_r_submission.csv")

Submission result: `0.08950`